In [ ]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import xlsxwriter
import warnings
warnings.simplefilter('ignore')

In [ ]:
curr_report_path = input('This quarter report: ') #
prev_report_path = input('Last quarter report: ') #

In [ ]:
filename = input("give the output file a name (eg. q3_master_file): ") #Q4_master_natural_persons

In [ ]:
curr_report = pd.read_excel(curr_report_path, sheet_name=None)
prev_report = pd.read_excel(prev_report_path, sheet_name=None)

In [ ]:
# create text file for links used
path = filename + '.txt'
with open(path, 'w') as f:
    f.write('Current Report path: ' + curr_report_path)
    f.write('\nPrevious Report path: ' + prev_report_path)
f.close()

In [ ]:
sheet_list = list(curr_report.keys())

df_dict = {}

for sheet in sheet_list:
    
    curr_df = curr_report.get(sheet)
    prev_df = prev_report.get(sheet)
    
    if (curr_df is None) or (prev_df is None): 
        
        df_dict[sheet] = curr_df
        break
    
    else: 
        # create column for identifying key
        df_curr['key'] = curr_df['Party ID'] + curr_df['Inconsistency'] + 
        df_prev['key'] = prev_df['Party ID'] + prev_df['Inconsistency']
    
        # identify prev_report records that are not resolved 
        curr_parties = df_curr['key'].tolist()
        prev_parties = df_prev['key'].tolist()
    
        # keep prev_report records that are still pending
        # edit: want to remove prev_report records if they are still pending (but keep remarks/batch id/fill-in fields)
        # idea is to replace old CRM information with new CRM information
        # list of parties that appear in both previous and current report (unresolved or exceptions)
        both_parties = set(prev_parties).intersection(set(curr_parties))
        new_parties = set(curr_parties).difference(set(prev_parties))
    
        #curr_df.loc[curr_df['key'].isin(prev_unresolved), 'Remarks'] = prev_df[prev_df['exist in current']==True]
    
        # remove curr_report records that are already present in prev_report
        # edit: KEEP curr_report records that are in prev_report
        df_curr = df_curr[(df_curr['key'].isin(both_parties)) | (df_curr['key'].isin(new_parties))]
    
        result_df = pd.concat([df_prev[df_prev['key'].isin(both_parties)], df_curr[df_curr['key'].isin(new_parties)]])
        result_df = result_df.drop(columns=['key'])
    
        df_dict[sheet] = result_df

In [ ]:
fin_sheet_list = list(df_dict.keys())

writer = ExcelWriter(filename + ".xlsx", mode='w', engine='xlsxwriter')

for sheet in fin_sheet_list:
    result = df_dict.get(sheet)
    result.to_excel(writer, sheet, index=False)
    
    workbook = writer.book
    format = workbook.add_format({'bg_color': '#FDE9D9'}) # use #FDE9D9
    worksheet = writer.sheets[sheet]
    
    # select columns to colour
    # note: at time of running, test reports only have "status of cleanup" and "completion date" as common columns
    cols = ['Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception']
    
    # apply colour to each column
    for col in cols:
        col_num = result.columns.get_loc(col)
        worksheet.set_column(col_num, col_num, 20, format)
    

writer.save()

In [ ]:
print('Completed.')